In [21]:
import sys
sys.path.append("..") 
from utils.surface_gases import *

In [22]:
COUPLER_options = {}

COUPLER_options['Phi_global'] = 1.0e-9
COUPLER_options['T_outgas'] = 3000.0

COUPLER_options['radius'] = 6.371e6
COUPLER_options['mass']   = 5.972e24
COUPLER_options['gravity'] = 6.67e-11 * COUPLER_options['mass'] / (COUPLER_options['radius']**2)
COUPLER_options['hydrogen_earth_oceans'] = 5.0
COUPLER_options['CH_ratio'] = 2.0
COUPLER_options['fO2_shift_IW'] = 5.0
COUPLER_options['nitrogen_ppmw'] = 2.0
COUPLER_options['planet_coresize'] = 0.55

solvepp_vols = ['H2O', 'CO2', 'N2', 'H2', 'CO', 'CH4']
for v in solvepp_vols:
    COUPLER_options[v+"_included"] = 0
    COUPLER_options[v+"_pa"] = 0.0

COUPLER_options["H2O_included"] = 1
COUPLER_options["CO2_included"] = 1
COUPLER_options["N2_included"]  = 1

COUPLER_options["H2O_pa"] = 1.0e5
# COUPLER_options["CO2_pa"] = 7.3e5



In [23]:
COUPLER_options["mantle_mass"] = calc_mantle_mass(COUPLER_options)

In [24]:
# solvepp_target = solvepp_get_target_from_params(COUPLER_options)
solvepp_target = solvepp_get_target_from_pressures(COUPLER_options)

print("Targets:")
print(solvepp_target)

{'H2O': 1.0, 'CO2': 0.0, 'N2': 0.0}
Targets:
{'H': 5.815920850604364e+17, 'C': 0.0, 'N': 0.0}


In [29]:
p_d = solvepp_equilibrium_atmosphere(solvepp_target, COUPLER_options)

print(p_d)

p_bar = {}
for v in volatile_species:
    p_bar[v] = p_d[v+"_pa"]*1.0e-5 # convert to bar

mubar = 0.0
ptot = 0.0
print("Partial pressures")
for v in solvepp_vols:
    print("    %s : %.1f bar"%(v, p_bar[v]))
    ptot += p_bar[v]
    mubar += p_bar[v]*molar_mass[v]
mubar /= ptot

print("mubar =",mubar)

m_d = {}
for v in solvepp_vols:
    m_d[v] = p_bar[v]*1.0e5 * 4 * 3.14 * (molar_mass[v]/mubar) * COUPLER_options["radius"]**2.0 / COUPLER_options["gravity"]

print(solvepp_atmosphere_mass(p_bar, COUPLER_options))

print(" ")
print(m_d)


C_kg = (m_d["CO2"]/molar_mass["CO2"] + m_d["CO"]/molar_mass["CO"] + m_d["CH4"]/molar_mass["CH4"]) * molar_mass['C']
H_kg = (2*m_d["H2O"]/molar_mass["H2O"] + 2*m_d["H2"]/molar_mass["H2"] + 4*m_d["CH4"]/molar_mass["CH4"]) *molar_mass['H']
N_kg = m_d["N2"]

print(H_kg, C_kg, N_kg)


{'H2O_pa': 100000.0, 'H2O_kg_atm': 5.197501835570717e+18, 'H2O_kg_int': 2112158106461.001, 'CO2_pa': 0.0, 'CO2_kg_atm': 0.0, 'CO2_kg_int': 0.0, 'H2_pa': 0.0, 'H2_kg_atm': 0.0, 'H2_kg_int': 0.0, 'CH4_pa': 0.0, 'CH4_kg_atm': 0.0, 'CH4_kg_int': 0.0, 'CO_pa': 0.0, 'CO_kg_atm': 0.0, 'CO_kg_int': 0.0, 'N2_pa': 0.0, 'N2_kg_atm': 0.0, 'N2_kg_int': 0.0}
Partial pressures
    H2O : 1.0 bar
    CO2 : 0.0 bar
    N2 : 0.0 bar
    H2 : 0.0 bar
    CO : 0.0 bar
    CH4 : 0.0 bar
mubar = 0.01801528
{'H2O': 5.197501835570717e+18, 'CO2': 0.0, 'N2': 0.0, 'H': 5.815918487134418e+17, 'C': 0.0, 'N': 0.0}
 
{'H2O': 5.194866923642551e+18, 'CO2': 0.0, 'N2': 0.0, 'H2': 0.0, 'CO': 0.0, 'CH4': 0.0}
5.81331609503898e+17 0.0 0.0


In [30]:
H_mol = H_kg / (1.0e-3)
H_oc  = H_mol / (7.69e22 * 2.0)


print(H_oc)
print("C/H = %.4f" % (C_kg/H_kg))

0.0037797893985949153
C/H = 0.0000
